In [27]:
from TwitterAPI import TwitterAPI
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from pytz import timezone

# Functions

In [56]:
def get_twitter():
    """ 
    Generates twitter connection
    """
    
    #Tokens
    consumer_key='nDx2bQS2DVdXEZd0b8MwaEG2L'
    consumer_secret='BeA1VuBq7hyi01fo4LFD13YAcrDgeYxIJTQgzx4gg8H5fSc3FR'
    access_token='900568210424164352-pKLEmPfNcqyGi3q67pyK3bJYtc5G01S'
    access_token_secret='uGbegX9whUY0FjZS6tgIrDITVOM8Q6tKMODNVPG2sT9Aq'
    
    
    twitter = TwitterAPI(
                   consumer_key,
                   consumer_secret,
                   access_token,
                   access_token_secret)
    return twitter

def get_NumberofFollowers(screen_name):       #does not crash when Exception, but does not correct exceptions either.
    '''
    returns the number of friends the user has
    '''
    try:
        request=twitter.request('users/lookup',{'screen_name':screen_name})
        return request.json()[0]['followers_count']
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN    

def get_timeline(screen_name,count=200): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count})
        return request.json()
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN
def get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = None): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count,
                                                            'exclude_replies':exclude_replies,'max_id':max_id})
        tweet_list = request.json()
        yesterday_DT = datetime.now(timezone('US/Central'))-timedelta(days=1)
        todaysTweets = []
        if tweet_list == []:
            return todaysTweets
        elif not (strToChicago_DT(tweet_list[-1]['created_at']) > yesterday_DT): #last of the 200 was in the last 24h
            todaysTweets =  [t for t in tweet_list if strToChicago_DT(t['created_at']) > yesterday_DT]
            return todaysTweets
        else:
            todaysTweets = tweet_list.copy()
            max_id = todaysTweets[-1]['id']
            print(max_id)
            todaysTweets += get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = max_id)[1:]
            return todaysTweets
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN

## Helper functions

In [37]:
def strToChicago_DT(string):
    UTC_DT = datetime.strptime(string,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo = timezone('UTC'))
    Chicago_DT = UTC_DT.astimezone(timezone('US/Central'))
    return Chicago_DT

# Import Accounts

In [5]:
accounts_df = pd.read_csv('Data/accounts.csv',sep=";")

# Collect Data

In [6]:
twitter = get_twitter()

In [13]:
accounts = accounts_df['Account'] #pd series
todaysTweets = pd.DataFrame(accounts, columns=['Account'])
date=str(datetime.now().date())
todaysTweets[date] = np.NaN

In [22]:
todaysTweets[date] = todaysTweets['Account'].apply(get_TodaysTweets)

In [24]:
todaysTweets.loc[todaysTweets[date]==-1]

,Account,2018-02-23
22,FoxNews,-1
24,Reuters,-1
25,el_pais,-1
87,thehill,-1
167,business,-1
170,TV_TradingIdeas,-1
176,BigCheds,-1
183,TV_TradingIdeas,-1


In [38]:
example = get_TodaysTweets('edballest')

In [55]:
l1 = [1,2,3]
l2 = [4,5]
l1[1:]

[2, 3]

In [ ]:
tweetdate = todaysTweets.loc[24,date][0]['created_at']
print(tweetdate)
tweetdate = 'Wed Feb 21 05:20:12 +0000 2018'
datetime.strptime(tweetdate,'%a %b %d %H:%M:%S +0000 %Y').date() == datetime.now().date()


In [ ]:
ne = todaysTweets[date].apply(lambda l: not(l == []))
todaysTweets.loc[ne,date].apply(lambda l: l[-1]['created_at'])

In [57]:
e = get_TodaysTweets('FoxNews')
print(e[199]['id'])
print(e[200]['id'])

967028098091929603
967028098091929603
967027983084081152


In [ ]:
e[0].keys()

In [ ]:
e[0]['coordinates']

In [ ]:
todaysTweets.loc[pd.isnull(todaysFollowers[date])] #check nulls

In [ ]:
accounts = accounts_df['Account'] #pd series
todaysFollowers = pd.DataFrame(accounts, columns=['Account'])
date=str(datetime.now().date())
# date = '2018-02-13' #for when I execute it after midnight
todaysFollowers[date] = np.NaN

In [ ]:
todaysFollowers[date] = todaysFollowers['Account'].apply(get_NumberofFollowers)

In [ ]:
todaysFollowers.head()

In [ ]:
todaysFollowers.loc[pd.isnull(todaysFollowers[date])] #check nulls

# Save Data

In [ ]:
todaysFollowers.to_csv('Data/todaysFollowers_'+date+'.csv',sep=';',index=False)

# Add to Historic Data

In [ ]:
historicData = pd.read_csv('Data/todaysFollowers_all.csv',sep = ';') #load historic

In [ ]:
historicData = pd.concat([historicData,todaysFollowers[date]],axis=1) #add todays column

In [ ]:
historicData.head()

In [ ]:
historicData.to_csv('Data/todaysFollowers_all.csv',sep=';',index=False) #save